In [1]:
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

%pwd

'/home/cseadmin/mhy/oyxd/capstone_project/GCRN/METRLA'

In [2]:
store=pd.HDFStore("./metr-la.h5")
df_data=store["df"]
index_list=df_data.index
ts=index_list[0]

def encode_timestamp(ts):
    weekday_embedding = [np.sin(2 * np.pi * ts.weekday()/6.0), np.cos(2 * np.pi * ts.weekday()/6.0)]
    hour_embedding = [np.sin(2 * np.pi * ts.hour/23.0), np.cos(2 * np.pi * ts.hour/23.0)]
    holiday_embedding=[1.0 if ts.weekday()>4 else 0.0]
    
    embedding=weekday_embedding+hour_embedding+holiday_embedding
    return embedding

time_embeddings=[]
for ts in index_list:
    time_embeddings.append(encode_timestamp(ts))
    
time_embeddings=np.array(time_embeddings)
# time_embeddings=np.broadcast_to(embeddings[:, np.newaxis, :], (embeddings.shape[0], 207, embeddings.shape[1]))
# np.save("./te_metrla.npy", time_embeddings)
time_embeddings.shape


(34272, 5)

In [3]:
import networkx as nx
from node2vec import Node2Vec
df = pd.read_csv('W_metrla.csv')
graph = nx.DiGraph(df.values)
node2vec = Node2Vec(graph, dimensions=2, walk_length=30, num_walks=200, workers=4)
model = node2vec.fit(window=10, min_count=1, batch_words=4)  
df_se = pd.DataFrame(model.wv.vectors, index=model.wv.index_to_key)
df_se.sort_index(inplace=True)
# np.save("./se_metrla.npy", df_se.values)
spatial_embeddings = df_se.values
spatial_embeddings.shape

Computing transition probabilities:   0%|          | 0/207 [00:00<?, ?it/s]

Generating walks (CPU: 4): 100%|██████████| 50/50 [00:15<00:00,  3.25it/s]


(207, 2)

In [4]:
spatial_embeddings = df_se.values.reshape(1,-1)
spatial_embeddings=np.broadcast_to(spatial_embeddings[:, :], (time_embeddings.shape[0], spatial_embeddings.shape[1]))
spatial_embeddings.shape

(34272, 414)

In [5]:
data_embedded=np.concatenate((spatial_embeddings, time_embeddings), axis=1)
data_embedded.shape

(34272, 419)

In [6]:
np.save("./METRLA_STembedded.npy", data_embedded)
np.save("./METRLA_Tembedded.npy", time_embeddings)
np.save("./METRLA_Sembedded.npy", spatial_embeddings)